#Data Augmentation Iteration 2

Install opendatasets

In [ ]:
!pip install opendatasets

Open the dataset from Kaggle. Requires username and key, see README.md for more details

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: johnnyplays
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset


Delete redundant image folder

In [ ]:
import shutil
shutil.rmtree("/content/face-expression-recognition-dataset/images/images")

Establish training and testing set

In [ ]:
train_set = "/content/face-expression-recognition-dataset/images/train"
test_set = "/content/face-expression-recognition-dataset/images/validation"

Function to create dataframe

In [ ]:
def dataframe(direct):
    image_paths = []
    labels = []
    for label in os.listdir(direct):
        for filename in os.listdir(os.path.join(direct, label)):
            image_paths.append(os.path.join(direct, label, filename))
            labels.append(label)

            # Check if it's an original image (not translated)
            if not filename.endswith("_translated.jpg"):
                # Add translated version if it exists
                translated_filename = f"{filename[:-4]}_translated.jpg"
                translated_path = os.path.join(direct, label, translated_filename)
                if os.path.exists(translated_path):
                    image_paths.append(translated_path)
                    labels.append(label)  # Same label as original

        print(label, "completed")
    return image_paths, labels

Create dataframe for training set

In [ ]:
import pandas as pd
import os

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = dataframe(train_set)

neutral completed
disgust completed
sad completed
fear completed
surprise completed
happy completed
angry completed


Create dataframe for testing set

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = dataframe(test_set)

neutral completed
disgust completed
sad completed
fear completed
surprise completed
happy completed
angry completed


Implement the translation function on training set

In [ ]:
import numpy as np
import cv2
from tqdm import tqdm  # For progress bar

# Assuming your training data is stored in a DataFrame called 'train'
# with columns 'image' and 'label'

# Get the count of samples in each class
label_counts = train['label'].value_counts()

# Find the maximum class size
max_class_size = label_counts.max()

print(f"Maximum class size: {max_class_size}")

Maximum class size: 7164


In [ ]:
def translate_image(image, shift_x, shift_y):
    """Translates an image by the given shift values."""
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    translated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return translated_image

training_dir = "/content/face-expression-recognition-dataset/images/train"  # Your training data path
shift_range = 5  # Maximum translation in pixels (adjust as needed)

for emotion_folder in os.listdir(training_dir):
    emotion_path = os.path.join(training_dir, emotion_folder)
    num_samples = len(os.listdir(emotion_path))  # Current samples in class

    # Calculate how many more samples are needed
    samples_needed = max_class_size - num_samples

    if samples_needed > 0:  # Augment if needed
        print(f"Augmenting {emotion_folder} with {samples_needed} samples")

        # Get a list of images in the current class
        image_files = os.listdir(emotion_path)

        # Augment and save new images
        for _ in range(samples_needed):
            # Randomly select an image to augment
            image_file = np.random.choice(image_files)
            image_path = os.path.join(emotion_path, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Generate random translation values
            shift_x = np.random.randint(-shift_range, shift_range + 1)
            shift_y = np.random.randint(-shift_range, shift_range + 1)

            # Translate the image
            translated_image = translate_image(image, shift_x, shift_y)

            # Save the augmented image
            new_filename = f"{image_file[:-4]}_translated_{_}.jpg"  # Unique name
            new_image_path = os.path.join(emotion_path, new_filename)
            cv2.imwrite(new_image_path, translated_image)

print("Data augmentation completed.")

Augmenting neutral with 2182 samples
Augmenting disgust with 6728 samples
Augmenting sad with 2226 samples
Augmenting fear with 3061 samples
Augmenting surprise with 3959 samples
Augmenting angry with 3171 samples
Data augmentation completed.
